# Predicting The Stock Market
------
In this project, We'll be working with data from the [S&P500 Index](https://www.kaggle.com/samaxtech/sp500-index-data). The S&P500 is a stock market index. 

# Introduction to Data

The columns of the dataset are:

- Date -- The date of the record.
- Open -- The opening price of the day (when trading starts).
- High -- The highest trade price during the day.
- Low -- The lowest trade price during the day.
- Close -- The closing price for the day (when trading is finished).
- Volume -- The number of shares traded.
- Adj Close -- The daily closing price, adjusted retroactively to include any corporate actions. 


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('sphist.csv')

In [2]:
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2015-12-07,2090.419922,2090.419922,2066.780029,2077.070068,4.043820e+09,2077.070068
1,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941
2,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117
3,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010
4,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16590 entries, 0 to 16589
Data columns (total 7 columns):
Date         16590 non-null object
Open         16590 non-null float64
High         16590 non-null float64
Low          16590 non-null float64
Close        16590 non-null float64
Volume       16590 non-null float64
Adj Close    16590 non-null float64
dtypes: float64(6), object(1)
memory usage: 907.4+ KB


In [4]:
# Convert the Date column to a Pandas date type
df['Date']= pd.to_datetime(df['Date'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16590 entries, 0 to 16589
Data columns (total 7 columns):
Date         16590 non-null datetime64[ns]
Open         16590 non-null float64
High         16590 non-null float64
Low          16590 non-null float64
Close        16590 non-null float64
Volume       16590 non-null float64
Adj Close    16590 non-null float64
dtypes: datetime64[ns](1), float64(6)
memory usage: 907.4 KB


In [6]:
# sort date value in ascending

df= df.sort_values('Date')


In [7]:
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close
16589,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66
16588,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85
16587,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93
16586,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98
16585,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08


Datasets taken from the stock market need to be handled differently than datasets from other sectors when it comes time to make predictions. 

In a normal machine learning exercise, we treat each row as independent. 
Stock market data is sequential, and each observation comes a day after the previous observation. 
Thus, the observations are not all independent, and you can't treat them as such.

The time series nature of the data means that can generate indicators to make our model more accurate.

Here are some indicators that are interesting to generate for each row:

- The average price from the past 5 days.
- The average price for the past 30 days.
- The average price for the past 365 days.
-  and the average price for the past 365 days.
- The standard deviation of the price over the past 5 days.
- The standard deviation of the price over the past 365 days.

"Days" means "trading days" -- so if you're computing the average of the past 5 days, it should be the 5 most recent dates before the current one. Assume that "price" means the Close column. 
Always be careful not to include the current price in these indicators!

Pandas has some time series tools that can help, including the **rolling function**, which will do most of the hard computation for you. Set the window equal to the number of trading days in the past you want to use to compute the indicators. This will add in NaN values for any row where there aren't enough historical trading days to do the computation.

In [8]:
# Calculating the means for trending days
# rolling provides rolling window calculations.
# Shift index by desired number of periods with an optional time freq

mean_5 = df['Close'].rolling(window = 5).mean().shift()
mean_30 = df['Close'].rolling(window=30).mean().shift()
mean_365 = df['Close'].rolling(window=365).mean().shift()

# Calculating Standard deviations

std_5 = df['Close'].rolling(window=5).std().shift()
std_30 = df['Close'].rolling(window=30).std().shift()
std_365 = df['Close'].rolling(window=365).std().shift()
    

In [9]:
# Add these indicators into the df dataframe
df['mean_5'] = mean_5
df['mean_30'] = mean_30
df['mean_365'] = mean_365
df['std_5'] = std_5
df['std_30'] = std_30
df['std_365'] = std_365

In [10]:
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close,mean_5,mean_30,mean_365,std_5,std_30,std_365
16589,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66,NaN,NaN,NaN,NaN,NaN,NaN
16588,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85,NaN,NaN,NaN,NaN,NaN,NaN
16587,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93,NaN,NaN,NaN,NaN,NaN,NaN
16586,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98,NaN,NaN,NaN,NaN,NaN,NaN
16585,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df = df.reset_index(drop = True)

In [12]:
df.head(10)

,Date,Open,High,Low,Close,Volume,Adj Close,mean_5,mean_30,mean_365,std_5,std_30,std_365
0,1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000.0,16.660000,NaN,NaN,NaN,NaN,NaN,NaN
1,1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000.0,16.850000,NaN,NaN,NaN,NaN,NaN,NaN
2,1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000.0,16.930000,NaN,NaN,NaN,NaN,NaN,NaN
3,1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000.0,16.980000,NaN,NaN,NaN,NaN,NaN,NaN
4,1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000.0,17.080000,NaN,NaN,NaN,NaN,NaN,NaN
5,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000.0,17.030001,16.900,NaN,NaN,0.157956,NaN,NaN
6,1950-01-11,17.090000,17.090000,17.090000,17.090000,2630000.0,17.090000,16.974,NaN,NaN,0.089051,NaN,NaN
7,1950-01-12,16.760000,16.760000,16.760000,16.760000,2970000.0,16.760000,17.022,NaN,NaN,0.067602,NaN,NaN
8,1950-01-13,16.670000,16.670000,16.670000,16.670000,3330000.0,16.670000,16.988,NaN,NaN,0.134796,NaN,NaN
9,1950-01-16,16.719999,16.719999,16.719999,16.719999,1460000.0,16.719999,16.926,NaN,NaN,0.196545,NaN,NaN


In [13]:
df_final = df.dropna()

In [14]:
df_final.reset_index(drop = True)

,Date,Open,High,Low,Close,Volume,Adj Close,mean_5,mean_30,mean_365,std_5,std_30,std_365
0,1951-06-19,22.020000,22.020000,22.020000,22.020000,1.100000e+06,22.020000,21.800000,21.703333,19.447726,0.256223,0.473595,1.790253
1,1951-06-20,21.910000,21.910000,21.910000,21.910000,1.120000e+06,21.910000,21.900000,21.683000,19.462411,0.213659,0.444648,1.789307
2,1951-06-21,21.780001,21.780001,21.780001,21.780001,1.100000e+06,21.780001,21.972000,21.659667,19.476274,0.092574,0.411452,1.788613
3,1951-06-22,21.549999,21.549999,21.549999,21.549999,1.340000e+06,21.549999,21.960000,21.631000,19.489562,0.115108,0.368514,1.787659
4,1951-06-25,21.290001,21.290001,21.290001,21.290001,2.440000e+06,21.290001,21.862000,21.599000,19.502082,0.204132,0.329130,1.786038
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16220,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883,2087.024023,2073.984998,2035.531178,3.916109,24.654181,64.370261
16221,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010,2090.231982,2076.283993,2035.914082,7.956808,23.970453,64.352527
16222,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117,2088.306006,2077.908659,2036.234356,9.333599,22.378095,64.277554
16223,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941,2080.456006,2078.931331,2036.507343,19.599946,20.183769,64.121622


Generate two new dataframes to use in making our algorithm. 
- train should contain any rows in the data with a date less than 2013-01-01. 
- test should contain any rows with a date greater than or equal to 2013-01-01.

In [30]:
# train and test dataset
train = df_final[df_final['Date'] < '2013-01-01']
test = df_final[df_final['Date'] >= '2013-01-01']

In [32]:
train.tail()


,Date,Open,High,Low,Close,Volume,Adj Close,mean_5,mean_30,mean_365,std_5,std_30,std_365
15846,2012-12-24,1430.150024,1430.150024,1424.660034,1426.660034,1.248960e+09,1426.660034,1437.360010,1405.926001,1326.114028,7.622009,24.746569,89.830647
15847,2012-12-26,1426.660034,1429.420044,1416.430054,1419.829956,2.285030e+09,1419.829956,1436.620019,1407.486336,1326.412494,8.589693,24.520427,89.983530
15848,2012-12-27,1419.829956,1422.800049,1401.800049,1418.099976,2.830180e+09,1418.099976,1431.228003,1408.813000,1326.716494,9.058684,24.055972,90.111444
15849,2012-12-28,1418.099976,1418.099976,1401.579956,1402.430054,2.426680e+09,1402.430054,1427.685986,1410.265332,1326.995836,10.208568,23.215373,90.236516
15850,2012-12-31,1402.430054,1426.739990,1398.109985,1426.189941,3.204330e+09,1426.189941,1419.434009,1411.830001,1327.261562,10.701861,20.858522,90.315637


In [33]:
test.head()

,Date,Open,High,Low,Close,Volume,Adj Close,mean_5,mean_30,mean_365,std_5,std_30,std_365
15851,2013-01-02,1426.189941,1462.430054,1426.189941,1462.420044,4.202600e+09,1462.420044,1418.641992,1414.258667,1327.534055,9.820801,17.834740,90.463948
15852,2013-01-03,1462.420044,1465.469971,1455.530029,1459.369995,3.829730e+09,1459.369995,1425.793994,1417.676668,1327.908247,22.261321,16.852563,90.738976
15853,2013-01-04,1459.369995,1467.939941,1458.989990,1466.469971,3.424290e+09,1466.469971,1433.702002,1420.092668,1328.224877,26.274326,17.470824,90.995857
15854,2013-01-07,1466.469971,1466.469971,1456.619995,1461.890015,3.304970e+09,1461.890015,1443.376001,1422.714665,1328.557617,27.945242,18.339803,91.279049
15855,2013-01-08,1461.890015,1461.890015,1451.640015,1457.150024,3.601600e+09,1457.150024,1455.267993,1425.076664,1328.898603,16.453319,18.678333,91.544368


# Linear Regression 1
Now, test the acuracy by performing Linear Regression
- Define an error metric
- Train a model using the train data
- Make predictions on the test data.

It's recommended to use Mean Absolute Error, also called MAE, as an error metric, because it will show you how "close" you were to the price in intuitive terms. Mean Squared Error, or MSE, is an alternative that is more commonly used, but makes it harder to intuitively tell how far off you are from the true price because it squares the error.


In [39]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

# Pick an error metric
features = [x for x in train.columns if x.startswith("mean") or x.startswith("std")]
target = 'Close'

#Initialize an instance of the LinearRegression class.
lr = LinearRegression()

# Train a linear regression model, using the train Dataframe
lr.fit(train[features], train[target])

# Make predictions for the Close column of the test data
prediction = lr.predict(test[features])

# Error Calculation 
rmse = mean_squared_error(prediction, test['Close']) ** 0.5
mae = mean_absolute_error(prediction, test[target])

print("The Root mean squared erros is:", rmse)
print("The mean absolute erros is:", mae)

The Root mean squared erros is: 22.203881310230823
The mean absolute erros is: 16.214387095798973


score() function returns the coefficient of determination R^2 of the prediction


In [41]:
lr.score(train[features],train[target])

0.99952355115572

# Adding New Indicators

- The ratio between the average price for the past 5 days, and the average price for the past 30 days.
- The ratio between the standard deviation for the past 5 days, and the standard deviation for the past 30 days.

In [42]:
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close,mean_5,mean_30,mean_365,std_5,std_30,std_365
0,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66,NaN,NaN,NaN,NaN,NaN,NaN
1,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85,NaN,NaN,NaN,NaN,NaN,NaN
2,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93,NaN,NaN,NaN,NaN,NaN,NaN
3,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98,NaN,NaN,NaN,NaN,NaN,NaN
4,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
df['mean_ratio'] = df['mean_30']/df['mean_5']
df['std_ratio'] = df['std_30']/ df['std_5']


In [44]:
df_final2 = df.dropna()

In [46]:
df_final2.isnull().sum()

Date          0
Open          0
High          0
Low           0
Close         0
Volume        0
Adj Close     0
mean_5        0
mean_30       0
mean_365      0
std_5         0
std_30        0
std_365       0
mean_ratio    0
std_ratio     0
dtype: int64

# Linear Regression 2 

In [47]:
# Creating Train and Test dataset from df_final2

train = df_final2[df_final2['Date'] < '2013-01-01']
test = df_final2[df_final2['Date'] >= '2013-01-01']

In [48]:
train.tail()

,Date,Open,High,Low,Close,Volume,Adj Close,mean_5,mean_30,mean_365,std_5,std_30,std_365,mean_ratio,std_ratio
15846,2012-12-24,1430.150024,1430.150024,1424.660034,1426.660034,1.248960e+09,1426.660034,1437.360010,1405.926001,1326.114028,7.622009,24.746569,89.830647,0.978131,3.246725
15847,2012-12-26,1426.660034,1429.420044,1416.430054,1419.829956,2.285030e+09,1419.829956,1436.620019,1407.486336,1326.412494,8.589693,24.520427,89.983530,0.979721,2.854634
15848,2012-12-27,1419.829956,1422.800049,1401.800049,1418.099976,2.830180e+09,1418.099976,1431.228003,1408.813000,1326.716494,9.058684,24.055972,90.111444,0.984339,2.655570
15849,2012-12-28,1418.099976,1418.099976,1401.579956,1402.430054,2.426680e+09,1402.430054,1427.685986,1410.265332,1326.995836,10.208568,23.215373,90.236516,0.987798,2.274107
15850,2012-12-31,1402.430054,1426.739990,1398.109985,1426.189941,3.204330e+09,1426.189941,1419.434009,1411.830001,1327.261562,10.701861,20.858522,90.315637,0.994643,1.949056


In [49]:
test.head()

,Date,Open,High,Low,Close,Volume,Adj Close,mean_5,mean_30,mean_365,std_5,std_30,std_365,mean_ratio,std_ratio
15851,2013-01-02,1426.189941,1462.430054,1426.189941,1462.420044,4.202600e+09,1462.420044,1418.641992,1414.258667,1327.534055,9.820801,17.834740,90.463948,0.996910,1.816017
15852,2013-01-03,1462.420044,1465.469971,1455.530029,1459.369995,3.829730e+09,1459.369995,1425.793994,1417.676668,1327.908247,22.261321,16.852563,90.738976,0.994307,0.757033
15853,2013-01-04,1459.369995,1467.939941,1458.989990,1466.469971,3.424290e+09,1466.469971,1433.702002,1420.092668,1328.224877,26.274326,17.470824,90.995857,0.990508,0.664939
15854,2013-01-07,1466.469971,1466.469971,1456.619995,1461.890015,3.304970e+09,1461.890015,1443.376001,1422.714665,1328.557617,27.945242,18.339803,91.279049,0.985685,0.656276
15855,2013-01-08,1461.890015,1461.890015,1451.640015,1457.150024,3.601600e+09,1457.150024,1455.267993,1425.076664,1328.898603,16.453319,18.678333,91.544368,0.979254,1.135232


In [51]:
# Pick an error metric
features = [x for x in train.columns if x.startswith("mean") or x.startswith("std")]
target = 'Close'

#Initialize an instance of the LinearRegression class.
lr = LinearRegression()

# Train a linear regression model, using the train Dataframe
lr.fit(train[features], train[target])

# Make predictions for the Close column of the test data
prediction = lr.predict(test[features])

# Error Calculation 
rmse = np.sqrt(mean_squared_error(prediction, test['Close']))
mae = mean_absolute_error(prediction, test[target])

print("The Root mean squared erros is:", rmse)
print("The mean absolute erros is:", mae)

The Root mean squared erros is: 22.179656330510486
The mean absolute erros is: 16.193221022480042


In [52]:
lr.score(train[features], train[target])

0.9995240056730667

Here we can see MAE value lowerd from 16.21 to 16.19. By adding 2 indicators we could improve accuracy.